In [1]:
import os
import dgl
import numpy as np

(g,), _ = dgl.load_graphs(os.path.join('/data/gangda/graph_engine', 'ogbn-products', 'dgl_data_processed'))
g

Graph(num_nodes=2449029, num_edges=123718152,
      ndata_schemes={}
      edata_schemes={'w': Scheme(shape=(), dtype=torch.float32)})

In [2]:
from ogb.nodeproppred import DglNodePropPredDataset

og, _ = DglNodePropPredDataset(name='ogbn-products', root='/data/gangda/dgl')[0]
og

Graph(num_nodes=2449029, num_edges=123718280,
      ndata_schemes={'feat': Scheme(shape=(100,), dtype=torch.float32)}
      edata_schemes={})

In [14]:
g = dgl.to_bidirected(og, copy_ndata=True)
g

Graph(num_nodes=2449029, num_edges=123718152,
      ndata_schemes={'feat': Scheme(shape=(100,), dtype=torch.float32)}
      edata_schemes={})

In [7]:
import torch
from torch_geometric.loader import NeighborLoader
import sklearn
from ogb.nodeproppred import DglNodePropPredDataset, PygNodePropPredDataset

batch_index = torch.tensor([10, 30, 50, 100, 900, 2000, 3030])

d = PygNodePropPredDataset('ogbn-products', root='/data/gangda/ogb', )
data = d[0]
data.n_id = torch.arange(data.num_nodes)
train_loader = NeighborLoader(
    data,
    input_nodes=batch_index,
    num_neighbors=[-1, -1],
    batch_size=batch_index.shape[0],
)

batch = next(iter(train_loader))

In [133]:
d.get_idx_split()

{'train': tensor([     0,      1,      2,  ..., 196612, 196613, 196614]),
 'valid': tensor([196615, 196616, 196617,  ..., 235935, 235936, 235937]),
 'test': tensor([ 235938,  235939,  235940,  ..., 2449026, 2449027, 2449028])}

In [22]:
ids = torch.tensor([68098,  47938, 174697,  69531,  26784, 127843,  59809, 143481,
       133705,  57786, 175932,  39531,  62559, 109486,  82512,   6551,
       138786,   5591, 151444,  48847,  99446, 180400, 189195,  49379,
        81849, 126478, 157139,   7077,  39650,  31071,  32633,  74105])
data.y[ids].view(-1)

tensor([ 4,  8,  4, 13,  9,  1, 13,  2, 21,  4, 12,  0,  1,  4,  6,  4,  7,  3,
         4,  0, 20,  5, 13,  4,  8,  3, 11,  4,  0,  4,  0,  4])

In [12]:
ppr_matrix = torch.load('/home/gangda/workspace/graph_engine/intermediate/{}_ppr_matrix.pt'.format('ogbn-products'))
ppr_matrix

tensor([[      0,  152857,  194591,  ...,  186457, 2034637,   35148],
        [      1,   89825,  151342,  ..., 1137752,  620866, 2322836],
        [      2,  488076, 1598820,  ..., 2017855,  442068, 1514525],
        ...,
        [2449026,  149963,  148503,  ..., 1886107, 2210439, 2009035],
        [2449027,  739621,  306502,  ...,  188674,  307615,  706682],
        [2449028,  728426,   48117,  ..., 1815472, 1420498, 2088456]])

---

In [57]:
batch_index = torch.tensor([68278,  56297, 171084, 112197, 137846, 116634,  64054, 122823,
       130182, 130749, 101573, 181379,  80995,  65400, 100146,  28231,
        83514, 112254,  18295, 130482, 126851,  62836, 133229,  25039,
       100577, 186332, 146552, 121370,  90180,  63159, 111289,  66735,
        19869, 173109,  74218,  97549, 106080, 140589,  16716, 191455,
       142625, 116690, 147426, 128014, 132821, 148122,  21356, 163926,
       119421,  92288,  11791,  89883, 145127,  59196, 147800,  54319,
       115086,   9869, 168121,  30365, 127473, 126582, 141283,  30128,
       111256,  77757,  99639,  24041,  77827, 130194,   1357,  37468,
       126713, 132848,  75706,  81772, 132363,  79648, 137766, 135999,
       126736, 124132,  40255,   5408, 156097, 173491,   2298,  18299,
       131653,  21709, 166116,  16551,  30994,  82181,  52336,   2867,
       153327,  54619,  47261, 126564,  57008,  60411,  33624,  81860,
       113063,  84535, 164701,  24259, 125796, 116581, 159517,  37141,
       114751,  29191, 112262, 112737,  71829, 106366,  82779,  99975,
       139694,   6663,  36672, 104501, 100275,  36964,  61175, 186781])


True

---

In [80]:
from torch_geometric.utils import contains_self_loops, add_remaining_self_loops

edge_index = add_remaining_self_loops(data.edge_index)[0]

In [109]:
from torch_geometric.utils import subgraph
from torch_geometric.data import Data

bid = 56297
sub_edge_index = subgraph(ppr_matrix[bid], edge_index)[0]

In [110]:
nid = 56297
t1 = sub_edge_index[1][sub_edge_index[0] == nid].sort().values
print(t1)

t2 = torch.tensor([   4103,    6024,   14835,   15266,   33845,   45496,   47577,
         52230,   56297,   85589,   89074,   96216,  104642,  107580,
        107737,  109598,  113832,  114736,  118252,  118762,  121192,
        125055,  127652,  128963,  131524,  135064,  140450,  146023,
        148605,  156638,  168160,  174389,  174515,  182856,  186570,
        191124,  194799,  198617,  200109,  214503,  223017,  223675,
        225407,  234131,  241399,  247375,  262409,  282174,  296068,
        305177,  327663,  426181,  501695,  503106,  569922,  588973,
        620013,  638524,  691897,  724889,  737063,  744513,  776309,
        796176,  807686,  820170,  907525,  918789,  923198,  938812,
        951967, 1038198, 1084952, 1085746, 1117551, 1119142, 1137652,
       1216534, 1276948, 1293009, 1294461, 1309569, 1324367, 1324550,
       1329821, 1334306, 1401245, 1451856, 1478828, 1510416, 1550307,
       1602713, 1661171, 1682742, 1728780, 1762245, 1832007, 1837622,
       1851124, 1878484, 1915810, 1930661, 1942460, 1997879, 2044222,
       2149572, 2167740, 2191269, 2192159, 2256802, 2344245])

np.setdiff1d(t1, t2), np.intersect1d(t1, t2).shape[0], t1.shape[0], t2.shape[0]

tensor([   4103,    6024,   14835,   15266,   33845,   45496,   47577,   52230,
          56297,   85589,   89074,   91998,   96216,  104642,  107580,  107737,
         109598,  113832,  114736,  118252,  118762,  121192,  125055,  127652,
         128963,  131524,  135064,  140450,  146023,  148605,  156638,  168160,
         174389,  174515,  182856,  186570,  186628,  191124,  194799,  198617,
         200109,  214503,  223017,  223675,  225407,  234131,  241399,  247375,
         262409,  282174,  296068,  305177,  327663,  426181,  501695,  503106,
         569922,  588973,  620013,  638524,  691897,  724889,  737063,  744513,
         776309,  796176,  807686,  820170,  907525,  918789,  923198,  938812,
         951967, 1038198, 1084952, 1085746, 1117551, 1119142, 1137652, 1216534,
        1276948, 1293009, 1294461, 1309569, 1324367, 1324550, 1329821, 1334306,
        1401245, 1451856, 1478828, 1510416, 1550307, 1602713, 1661171, 1682742,
        1728780, 1762245, 1832007, 18376

(array([ 91998, 186628]), 111, 113, 111)

---

In [126]:
ns1 = ppr_matrix[130749]

ns2 = torch.tensor([   6222,    6251,    6268,    7296,   10588,   11145,   15417,
         16511,   20952,   21437,   21720,   25005,   32210,   32598,
         33255,   38843,   41492,   50061,   58896,   61115,   62735,
         64870,   65999,   70400,   72903,   74089,   75973,   76026,
         76832,   80122,   81675,   85569,   86718,   86970,   90106,
         90134,   92013,   97023,   97108,   98078,  107187,  108055,
        111736,  112638,  121554,  124241,  124370,  125624,  127373,
        129807,  129893,  130749,  142731,  145867,  146708,  150824,
        151435,  152372,  155996,  156405,  156767,  157809,  160665,
        161910,  167580,  170537,  171759,  173016,  173851,  175338,
        178624,  180264,  195225,  197482,  198567,  198696,  198942,
        203058,  214260,  218585,  228173,  228997,  341695,  383742,
        400165,  418829,  424121,  427091,  447866,  461709,  507888,
        698911,  798487,  836781,  892154, 1081300, 1084031, 1107592,
       1121145, 1135364, 1221334, 1258255, 1273891, 1293074, 1309694,
       1338520, 1372122, 1377538, 1380713, 1384349, 1453199, 1457487,
       1467081, 1474994, 1518215, 1542457, 1542741, 1548980, 1552649,
       1564253, 1653834, 1673178, 1717108, 1738934, 1751805, 1766540,
       1829955, 1837460, 1905512, 1914379, 1948321, 1948749, 2028565,
       2031083, 2073091, 2132533, 2176578, 2176934, 2190671, 2203444,
       2242442, 2243360, 2302481, 2343492, 2344023, 2354508, 2356640,
       2374541, 2389262, 2410762])

np.intersect1d(ns1, ns2).shape[0], ns1.shape[0], ns2.shape[0]

(133, 150, 150)

In [128]:
total_intersect = 147 + 120 + 144 + 133 + 126 + 125 + 139 + 128 + 128 + 133
total_num       = 150 + 120 + 150 + 150 + 150 + 150 + 150 + 150 + 150 + 150
print(f'{total_intersect/total_num:.3f}')

0.900


---

In [96]:
current_node = 7798
ours = sub_edge_index[1][sub_edge_index[0] == current_node]
# ours = torch.cat([torch.tensor([current_node]), ours])
ours.shape, ours

(torch.Size([0]), tensor([], dtype=torch.int64))

In [48]:
theirs = torch.tensor([  17446,   17467,   35854,   39891,   52454,   68098,   87180,
        110057,  122777,  139400,  151294,  163313,  165683,  170942,
        186811,  211284,  211636,  223473,  470411,  625242,  672850,
        715973,  787213,  801092,  829837,  903675, 1030175, 1197019,
       1284680, 1313296, 1325882, 1397585, 1479103, 1741776, 1922081,
       1985068, 2045759, 2086319, 2190390, 2232986, 2362467, 2417788,
       2441676])
theirs.shape

torch.Size([43])

In [51]:
np.intersect1d(ours, theirs).shape[0]

2

In [43]:
np.intersect1d(ppr_matrix[batch_index], batch.n_id).shape[0]

1046

In [6]:
from torch_sparse import SparseTensor

og, y = DglNodePropPredDataset(name='ogbn-products', root='/data/gangda/dgl')[0]
dataset = dict(X=og.ndata['feat'], y=y)
dataset['edge_index'] = torch.load('/home/gangda/workspace/graph_engine/data/ogbn-products-p2/dgl_edge_index.pt')
rowptr, col, _ = SparseTensor.from_edge_index(dataset['edge_index']).csr()

NameError: name 'torch' is not defined

In [33]:
bid = torch.tensor(10)
subset, inv = torch.cat([bid.unsqueeze(dim=0), ppr_matrix[bid]]).unique(return_inverse=True)
ego_index = inv[0]

subset, ego_index

(tensor([     10,    2428,    3785,    4015,    5555,    6560,    9181,   16782,
           18993,   19892,   21989,   23371,   32573,   35812,   39850,   43289,
           43800,   47466,   49270,   51779,   54453,   55178,   57560,   59483,
           68228,   76725,   88265,   90737,   92841,   96347,   98124,  102821,
          111062,  118078,  121638,  125403,  125559,  128263,  129128,  132055,
          148930,  149434,  153532,  163968,  164914,  165311,  167623,  168475,
          168952,  169721,  170486,  176703,  177773,  180425,  180485,  184788,
          184970,  186565,  187343,  189660,  193115,  196073,  196453,  200695,
          208672,  213938,  217341,  227457,  239076,  256497,  351947,  461788,
          467602,  517841,  552193,  559657,  565815,  603598,  613878,  614941,
          624900,  667619,  693329,  697337,  715845,  743221,  755881,  827803,
          830755,  833989,  918186,  928645,  960868, 1123270, 1165180, 1201781,
         1224592, 1285771, 1

In [44]:
from pyg_lib.sampler import subgraph as libsubgraph

edge_id = libsubgraph(rowptr, col, subset, return_edge_id=True)[-1]
sub_edge_index = dataset['edge_index'][:, edge_id]
subset2, local_edge_index = sub_edge_index.unique(return_inverse=True)

In [43]:
subset2.equal(subset)

True

In [58]:
from torch_geometric.utils import contains_self_loops, contains_isolated_nodes, is_undirected

print(contains_isolated_nodes(local_edge_index))
print(is_undirected(local_edge_index))
print(contains_self_loops(local_edge_index))

False
True
False


In [62]:
local_edge_index.shape

torch.Size([2, 2166])

In [56]:
data.x[sub_edge_index].equal(dataset['X'][subset][local_edge_index])

True

In [69]:
one_hop = sub_edge_index[1][sub_edge_index[0] == 10]
one_hop.shape

torch.Size([84])

In [79]:
batch.y[:batch.batch_size]

tensor([[7],
        [7],
        [4],
        [1],
        [0],
        [3],
        [0]])

In [82]:
from tqdm import tqdm

data_list = []
for i in tqdm(range(20)):
    data_list += torch.load(
        '/home/gangda/workspace/graph_engine/intermediate/{}_egograph_list_{}.pt'.format('ogbn-products', i))

len(data_list)

100%|██████████| 20/20 [14:52<00:00, 44.65s/it]


2449029

In [91]:
b_x, b_y = [], []

for bid in batch_index.tolist():
    bd = data_list[bid]
    b_x.append(bd.x[bd.ego_index])
    b_y.append(bd.y)

b_x = torch.stack(b_x)
b_y = torch.stack(b_y)
b_x.shape, b_y.shape

(torch.Size([7, 100]), torch.Size([7, 1]))

In [92]:
b_x.equal(data.x[batch_index]), b_y.equal(data.y[batch_index])

(True, True)

Data(num_nodes=2449029, edge_index=[2, 123718280], x=[2449029, 100], y=[2449029, 1], n_id=[2449029])